In [ ]:
import gspread
import os
import pandas as pd
from gspread_dataframe import set_with_dataframe
from google.oauth2 import service_account
from bs4 import BeautifulSoup
from urllib.request import urlopen,Request

In [ ]:
signos = ['aries','touro','gemeos','cancer','leao','virgem','libra','escorpiao','sagitario','capricornio','aquario','peixes']

In [ ]:
url = 'https://joaobidu.com.br/horoscopo/signos/previsao-'

In [ ]:
url_imagem = 'https://joaobidu.com.br/static/img/'

In [ ]:
lista_texto = {}
lista_imagem = {}
for signo in signos:
    resposta = Request(url+signo,headers={'User-Agent': 'Mozilla/5.0'})
    html = urlopen(resposta,timeout=10).read()
    conteudo = BeautifulSoup(html, 'html.parser')
    imagem = conteudo.p.img.get('src').replace('../static/img/','')
    texto = conteudo.p.get_text()
    lista_texto[signo] = texto.replace('\n','')
    lista_imagem[signo] = url_imagem+imagem

In [ ]:
df_signos_descricao = pd.DataFrame(list(lista_texto.items()),
                   columns=['Signos', 'Descrição'])

In [ ]:
df_signos_imagens = pd.DataFrame(list(lista_imagem.items()),
                   columns=['Signos', 'Imagem'])

In [ ]:
df_merge_signos = pd.merge(df_signos_descricao,df_signos_imagens, how='right', on = 'Signos')

In [ ]:
df_merge_signos

In [ ]:
scopes = ["https://www.googleapis.com/auth/spreadsheets",
          "https://www.googleapis.com/auth/drive"]

In [ ]:
json_file = 'chave/' + str(os.listdir('chave/')[0])

In [ ]:
credentials = service_account.Credentials.from_service_account_file(json_file)
scoped_credentials = credentials.with_scopes(scopes)
gc = gspread.authorize(scoped_credentials)

In [ ]:
plan_aux = gc.open("BotTelegram")
aba_aux = plan_aux.worksheet("Teste")

In [ ]:
aba_aux.update([df_merge_signos.columns.values.tolist()] + df_merge_signos.values.tolist())